In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_datasets as tfds

# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [2]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sta314 (takim). Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'normalization': {
          'values': [False, True]
        }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.001},
    'batch_size': {
        'value': 64},
    'kernel_size': {
          'value': '5x5'},
    'net_filter_size': {
          'value': 32},
    'net_n': {
          'value': 3},
    'reg_alpha': {
          'value': 0.0001}
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'value': 64},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'kernel_size': {'value': '5x5'},
                'learning_rate': {'value': 0.001},
                'net_filter_size': {'value': 32},
                'net_n': {'value': 3},
                'normalization': {'values': [False, True]},
                'reg_alpha': {'value': 0.0001}}}


In [6]:
# sweep_id = wandb.sweep(sweep_config, project="zayif-test")
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: zia6uq63
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/zia6uq63


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Lambda, Add, Input, GlobalAveragePooling2D, Flatten, Dense, Softmax
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2

def ResidualBlock(x, filter_size, is_switch_block, kernel_size, reg_alpha):

    # note that if is_switch_block true, it means that output will not be the same as the input
    # so while merging the residual connection, we need to adapt to it
    # this adaptation could be with a conv layer, or a simple downsampling + padding is enough.

    x_skip = x # save original input to the block

    if not is_switch_block:
        x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    else:
        x = Conv2D(filter_size, kernel_size=kernel_size, strides=(2, 2), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    x = BatchNormalization()(x)

    if is_switch_block: # takes every second element to half(v) spatial dimension and then adds padding to each side for matching filter (last) dimension
        x_skip = Lambda(lambda x: tf.pad(x[:, ::2, ::2, :], tf.constant([[0, 0,], [0, 0], [0, 0], [filter_size//4, filter_size//4]]), mode="CONSTANT"))(x_skip)

    x = Add()([x, x_skip])
    x = ReLU()(x)

    return x

def ResidualBlocks(x, filter_size, n, kernel_size, reg_alpha):
    for group in range(3): # a stack of 6n layers, 3×3 convolutions, feature maps of sizes {4fs, 2fs, fs}, 2n layers for each size
        for block in range(n):
            if group > 0 and block == 0: # double filter size
                filter_size *= 2
                is_switch_block = True
            else:
                is_switch_block = False
                
            x = ResidualBlock(x, filter_size, is_switch_block, kernel_size, reg_alpha)

    return x

def create_model(config):

    filter_size = config['net_filter_size']
    n = config['net_n']
    kernel_size = (3, 3) if config['kernel_size'] == '3x3' else (5, 5)

    reg_alpha = config['reg_alpha']

    inputs = Input(shape=(32, 32, 3))
    x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = ResidualBlocks(x, filter_size, n, kernel_size, reg_alpha)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(10)(x)
    outputs = Softmax()(x)

    model = Model(inputs, outputs, name=f"ResNet-{n*6+2}")
    return model

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        x_train_to_use = (x_train.astype('float32') / 255) if config['normalization'] else x_train
        x_test_to_use = (x_test.astype('float32') / 255) if config['normalization'] else x_test

        tf.keras.backend.clear_session()
        model = create_model(config)
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        history = model.fit(x_train_to_use, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )
        
        test_stats = model.evaluate(x_test_to_use, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [9]:
# wandb.agent(sweep_id, train, count=1)
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: kzmi7ixf with config:
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: 5x5
wandb: 	learning_rate: 0.001
wandb: 	net_filter_size: 32
wandb: 	net_n: 3
wandb: 	normalization: False
wandb: 	reg_alpha: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 43s 55ms/step - loss: 1.5146 - accuracy: 0.5039 - top@3_accuracy: 0.8132 - val_loss: 2.2335 - val_accuracy: 0.3938 - val_top@3_accuracy: 0.6902
Epoch 2/100
704/704 [==============================] - 41s 59ms/step - loss: 1.0518 - accuracy: 0.6811 - top@3_accuracy: 0.9136 - val_loss: 1.6175 - val_accuracy: 0.5570 - val_top@3_accuracy: 0.8396
Epoch 3/100
704/704 [==============================] - 45s 64ms/step - loss: 0.8836 - accuracy: 0.7471 - top@3_accuracy: 0.9406 - val_loss: 1.2906 - val_accuracy: 0.6200 - val_top@3_accuracy: 0.8860
Epoch 4/100
704/704 [==============================] - 49s 69ms/step - loss: 0.8057 - accuracy: 0.7818 - top@3_accuracy: 0.9527 - val_loss: 1.2015 - val_accuracy: 0.6606 - val_top@3_accuracy: 0.9194
Epoch 5/100
704/704 [==============================] - 51s 73ms/step - loss: 0.7546 - accuracy: 0.8069 - top@3_accuracy: 0.9604 - val_loss: 1.1133 - val_accuracy: 0.7000 - val_top@3_accuracy: 0.9138
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇█████████████
epoch/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▅▆▆▇▇▇▇▇▇██████████████████
epoch/val_accuracy,▁▄▅▅▆▅▆▇▇▆▇▇▆▇▅▇▇███▇█▆███▇▇
epoch/val_loss,█▅▃▃▂▄▃▂▁▄▂▂▃▂▆▂▂▁▂▂▃▂▇▂▂▂▂▃
epoch/val_top@3_accuracy,▁▅▆▇▇▆▇▇█▆▇██▇▇▇████▇█▆█████
test_acc,▁


wandb: Agent Starting Run: d6k90ykh with config:
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: 5x5
wandb: 	learning_rate: 0.001
wandb: 	net_filter_size: 32
wandb: 	net_n: 3
wandb: 	normalization: True
wandb: 	reg_alpha: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 57s 78ms/step - loss: 1.5845 - accuracy: 0.4740 - top@3_accuracy: 0.7990 - val_loss: 1.6862 - val_accuracy: 0.4486 - val_top@3_accuracy: 0.7876
Epoch 2/100
704/704 [==============================] - 56s 80ms/step - loss: 1.1001 - accuracy: 0.6572 - top@3_accuracy: 0.9039 - val_loss: 1.2539 - val_accuracy: 0.6010 - val_top@3_accuracy: 0.8834
Epoch 3/100
704/704 [==============================] - 57s 81ms/step - loss: 0.9113 - accuracy: 0.7316 - top@3_accuracy: 0.9362 - val_loss: 1.4985 - val_accuracy: 0.5502 - val_top@3_accuracy: 0.8290
Epoch 4/100
704/704 [==============================] - 58s 82ms/step - loss: 0.8147 - accuracy: 0.7745 - top@3_accuracy: 0.9503 - val_loss: 1.0144 - val_accuracy: 0.7020 - val_top@3_accuracy: 0.9196
Epoch 5/100
704/704 [==============================] - 58s 83ms/step - loss: 0.7482 - accuracy: 0.8034 - top@3_accuracy: 0.9600 - val_loss: 0.9639 - val_accuracy: 0.7380 - val_top@3_accuracy: 0.9328
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▄▅▅▆▆▆▇▇▇▇▇▇▇████████
epoch/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▅▆▆▇▇▇▇▇▇████████████
epoch/val_accuracy,▁▄▃▆▇▅▇▆▇▆██▇▅▇█▇▇█▇█▆
epoch/val_loss,█▄▆▂▂▅▂▄▂▄▁▁▂▇▂▂▃▄▂▄▄▅
epoch/val_top@3_accuracy,▁▅▃▆▇▆▇▇▇▇██▇▅▇█▇▇█▇█▇
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
